In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('../..'))) # access sibling directories
sys.path.append("C:\\Users\\lasse\\Documents\\GitHub\\BayesFlow")

from src.python.settings import summary_meta_diffusion, evidence_meta_diffusion
from src.python.helpers import load_simulated_rt_data, mask_inputs
from src.python.networks import HierarchicalInvariantNetwork, EvidentialNetwork

import numpy as np
from tensorflow.keras.experimental import CosineDecayRestarts
from tensorflow.keras.optimizers import Adam
from functools import partial

from bayesflow.trainers import ModelComparisonTrainer
from bayesflow.amortizers import MultiModelAmortizer 
from bayesflow.losses import log_loss

# Fine-tuning with 900 trials per person

## Load in data

In [2]:
levy_sims_folder = 'c:\\Users\\lasse\\documents\\hierarchical model comparison project\\data\\Levy_flight_application'

indices_900_filename = "train_indices_900_trials.npy"
datasets_900_filename = "train_datasets_900_trials.npy"

indices_900, datasets_900 = load_simulated_rt_data(levy_sims_folder, indices_900_filename, datasets_900_filename)

## Training

When only conducting fine-tuning: manually move/delete fine-tuning checkpoints in checkpoints folder so that training resumes from pretrained network.

In [3]:
summary_net = HierarchicalInvariantNetwork(summary_meta_diffusion)
evidence_net = EvidentialNetwork(evidence_meta_diffusion)
amortizer = MultiModelAmortizer(evidence_net, summary_net)

# Cosine Decay with Restarts
initial_lr = 0.00005 # Shrink LR by factor 10 for fine-tuning
first_decay_steps = 250
t_mul = 2
m_mul = 0.9
alpha = 0.2
lr_schedule_restart = CosineDecayRestarts(
    initial_lr, first_decay_steps, t_mul=t_mul, m_mul=m_mul, alpha=alpha)

# Checkpoint path for loading pretrained network and saving the final network
trial_folder = 'trial_4'
checkpoint_path = os.path.join(os.getcwd(), 'training_checkpoints', trial_folder)
#checkpoint_path = os.path.join(os.getcwd(), 'checkpoints', trial_folder) # rename checkpoint folder to solve the odd "UnknownError: Failed to rename" error that can happen during training

trainer = ModelComparisonTrainer(
    network=amortizer, 
    loss=partial(log_loss, kl_weight=0.25),
    optimizer=partial(Adam, lr_schedule_restart),
    checkpoint_path=checkpoint_path,
    skip_checks=True
    )

TRAINER INITIALIZATION: No generative model provided. Only offline learning mode is available!
Networks loaded from c:\Users\lasse\Dropbox\Uni Mannheim\M.Sc\4. Semester\Masterarbeit\Coding\notebooks\03_levy_flight_application\training_checkpoints\trial_4\ckpt-57


In [4]:
# Mask some training data so that training leads to a robust net that can handle missing data
epochs = 7

for epoch in range(epochs):
    datasets_900_masked = mask_inputs(datasets_900, missings_mean=28.5, missings_sd=13.5, missing_rts_equal_mean=True)
    losses = trainer.train_offline(epochs=1, batch_size=32, 
                               model_indices=indices_900, sim_data=datasets_900_masked)
    print(f"epoch {epoch} finished.")

Converting 8000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

epoch 0 finished.
Converting 8000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

epoch 1 finished.
Converting 8000 simulations to a TensorFlow data set...


...

epoch 4 finished.
Converting 8000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

epoch 5 finished.
Converting 8000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

epoch 6 finished.
